# Extract data from a Common Voice dataset by accent 

The purpose of this notebook is to extract all the Australian-accented speech samples from Common Voice English so that we can use the data to fine-tune a Whisper model on Australian-accented speech. 

The extraction is done _not just_ on speech samples having the accent `Australian English` but _additional_ contributor-specified accent descriptors such as e.g. `Sydney` or `Australian Eastern Seaboard`. 



## Import libraries 

In [24]:
import pandas as pd 
import matplotlib as mpl 
import numpy as np
from tqdm import tqdm
import re
import subprocess
import os
import shutil
import librosa

## Import the Common Voice `.tsv` file into a `pandas` dataframe

Here, we extract data from the TSV file, and use `pandas` to perform manipulations on the dataset, such as removing rows that do not contain accent metadata.

In [3]:
# we only want to read from the validated data, not the invalidated
# change this to where you have downloaded the CV dataset
# the datasets can be downloaded from Mozilla Data Collective 
# https://datacollective.mozillafoundation.org/datasets?q=common+voice
filePath = '/media/kathyreid/Elements/cv-datasets/cv-corpus-23.0-2025-09-05/en/validated.tsv'

In [4]:
# put it into a DataFrame 
df = pd.read_csv(filePath, sep='\t', low_memory=False)

In [5]:
df.columns

Index(['client_id', 'path', 'sentence_id', 'sentence', 'sentence_domain',
       'up_votes', 'down_votes', 'age', 'gender', 'accents', 'variant',
       'locale', 'segment'],
      dtype='object')

In [6]:
# rows that have accent metadata 
len(df[df['accents'].notna()])

1026637

In [7]:
# what proportion of rows have accent metadata compared with those that don't 
print(len(df[df['accents'].notna()]) / (len(df)) * 100)

55.14276659970566


In [8]:
# remove all the rows where accents are not given (NaN)
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html
# DataFrame.dropna(*, axis=0, how=_NoDefault.no_default, thresh=_NoDefault.no_default, subset=None, inplace=False)

df.dropna(axis='index', how='any', subset='accents', inplace=True)
len(df)

# this matches the above figure for rows that have accent metadata, so it's a good cross-check

1026637

In [9]:
# number of unique contributors to the dataset 
len(df['client_id'].unique())

18734

## Filter the dataset for accents which are just Australian or something to indicate Australian 

In [10]:
all_accents = df['accents']

In [11]:
# The original accent entries are given comma-delimited, e.g. `Non native speaker, German English`
# We want to turn this into a list of specific accents, the we can take only the unique values 

accents_list = []

for idx, accent_string in list(enumerate(all_accents)): 
    
    # this regex is from https://stackoverflow.com/questions/26633452/how-to-split-by-commas-that-are-not-within-parentheses
    accent_list = re.split(r',\s*(?![^()]*\))', accent_string)
    processed_accent_list = accent_list # we don't want to modify a list we're iterating over

    for idx_a, accent in list(enumerate(accent_list)): 
    
        # Trim any whitespace off the elements, because this makes matching on strings harder later on
        # Strings are immutable in Python, so we have to create another string
        processed_accent_list.remove(accent)
        stripped_accent = accent.strip() 
        processed_accent_list.append(stripped_accent)
        
        # Check for any empty strings and remove them - likely regex artefacts
        if not accent: 
            processed_accent_list.remove(accent)
            
        # Check for any non-Latin characters that we may want to investigate 
        # For example, if one of the accents is garbage or deliberate rubbish
        
        if not accent.isascii(): 
            print('flagging that accent: ', accent, ' is not ASCII encoded, may be in another language')
            
    accents_list.append(processed_accent_list)

flagging that accent:  Выраженный украинский акцент  is not ASCII encoded, may be in another language
flagging that accent:  Выраженный украинский акцент  is not ASCII encoded, may be in another language
flagging that accent:  Выраженный украинский акцент  is not ASCII encoded, may be in another language
flagging that accent:  Русский  is not ASCII encoded, may be in another language
flagging that accent:  Русский  is not ASCII encoded, may be in another language
flagging that accent:  Русский  is not ASCII encoded, may be in another language
flagging that accent:  Русский  is not ASCII encoded, may be in another language
flagging that accent:  Немного русский акцент произношения  is not ASCII encoded, may be in another language
flagging that accent:  съедание слов  is not ASCII encoded, may be in another language
flagging that accent:  Немного русский акцент произношения  is not ASCII encoded, may be in another language
flagging that accent:  съедание слов  is not ASCII encoded, may b

at this point, `accents_list` is a list of lists:

```
['Irish English']
['England English']
['Danish English', 'Transatlantic']
```
We want to turn it into a single list, then into a set - with only unique elements - each of the accents. 

In [12]:
flattened = [item for sublist in accents_list for item in sublist]

In [13]:
print(len(flattened))

1118507


In [14]:
# turn into a set 
unique_accents = list(set(flattened))

In [15]:
len(unique_accents)

831

In [16]:
#print(unique_accents)

## Figure out which of the accents above are Australian

This is a manual exercise for now .. 

* `Australian English`
* `General Australian`
* `lived with my grandfather for a time who has a mix between the australian and an english accent`
* `Hybrid Indian/Australian/American`
* `but spent a lot of time in australia and the US`
* `Hybrid United States and Australian`
* `South Australia`
* `Educated Australian Accent`
* `Sydney - middle eastern seaboard Australian`
* `Queenslandish`

For now we will only select rows with: 

* `Australian English`
* `General Australian`
* `South Australia`
* `Educated Australian Accent`
* `Sydney - middle eastern seaboard Australian`
* `Queenslandish`

to make it as "General Australian" as possible 

## Filter the dataframe by the Australian accents 

In [17]:
accents_to_match = ['Australian English', 'General Australian', 'South Australia', 'Sydney', 'Queensland']

In [18]:
# Using regex with the '|' (OR) operator
filtered_df = df[df['accents'].str.contains('|'.join(accents_to_match), na=False)]

In [19]:
len(filtered_df)
# 55757 rows

55757

## Identify how many unique contributors there are 

We do this so for example, it's not one speaker who has contributed 55k samples

In [20]:
len(filtered_df['client_id'].unique())

834

In [21]:
#filtered_df['client_id'].value_counts()
# this shows that there are about ten key contributors who have thousands of samples
# but not one who has say 20k samples
# so this should be OK for fine-tuning. 

# This does have implications for how much data ends up into train and test 
#    as we don't want to train on the same speakers we're testing on
#    - this is called "data leakage" - or "learning to the test"

## Add a `duration_ms` column and calculate audio duration

In [30]:
filtered_df['duration_ms'] = None

/tmp/ipykernel_1350737/1759776887.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['duration_ms'] = None


In [31]:
filtered_df.columns

Index(['client_id', 'path', 'sentence_id', 'sentence', 'sentence_domain',
       'up_votes', 'down_votes', 'age', 'gender', 'accents', 'variant',
       'locale', 'segment', 'duration_ms'],
      dtype='object')

In [32]:
filtered_df_copy = filtered_df.copy(deep=True)

In [35]:
audio_path = '/media/kathyreid/Elements/cv-datasets/cv-corpus-23.0-2025-09-05/en/clips/'

for index, row in tqdm(filtered_df_copy.iterrows(), total=len(filtered_df_copy), desc="Calculating audio duration..."):
    file_path = audio_path + row['path']
    duration = librosa.get_duration(path=file_path)
    # insert into dataframe 
    filtered_df.loc[index, 'duration_ms'] = duration

Calculating audio duration...: 100%|█████| 55757/55757 [02:05<00:00, 444.31it/s]


## Copy the matching audios across

I intend to use the [Mozilla.AI blueprint](https://mozilla-ai.github.io/speech-to-text-finetune/customization/) authored by [Kostis Saitas Zarkias](https://github.com/Kostis-S-Z). The format used in that blueprint is: 

```
datasets/
├── my_dataset/
│   ├── dataset.csv
│   └── audio_files/
│       ├── audio_1.wav
│       ├── audio_2.wav
│       ├── audio_3.wav
│       └── ...
```

and I want to match the format here. 

In [43]:
# define the file path to save the audio files
audio_fp_src = '/media/kathyreid/Elements/cv-datasets/cv-corpus-23.0-2025-09-05/en/clips/'
audio_fp_destination = 'datasets/commonvoice-v23_en-AU/audio_files/'

In [44]:
# iterate over the dataframe and output the audio files to match the structure above.
# make sure the directory structure exists first  - easier to do this on the command line

for index, row in tqdm(filtered_df.iterrows(), total=len(filtered_df), desc="Copying audios across to new dataset"):

    source_file = audio_fp_src + row['path']
    destination_file = audio_fp_destination + row['path']

    try:
        shutil.copyfile(source_file, destination_file)
        # we don't want to setn too much info to screen
        if index%1000 == 0: 
            print(f"File copied from {source_file} to {destination_file}")
    except shutil.SameFileError:
        print("Source and destination represent the same file.")
    except IsADirectoryError:
        print("Destination is a directory. Use shutil.copy() instead if destination is a folder.")
    except FileNotFoundError:
        print("Source file not found.")

Copying audios across to new dataset:   1%| | 299/55757 [00:02<04:43, 195.56it/s

File copied from /media/kathyreid/Elements/cv-datasets/cv-corpus-23.0-2025-09-05/en/clips/common_voice_en_38162284.mp3 to datasets/commonvoice-v23_en-AU/audio_files/common_voice_en_38162284.mp3


Copying audios across to new dataset:   1%| | 667/55757 [00:03<03:23, 270.97it/s

File copied from /media/kathyreid/Elements/cv-datasets/cv-corpus-23.0-2025-09-05/en/clips/common_voice_en_19667424.mp3 to datasets/commonvoice-v23_en-AU/audio_files/common_voice_en_19667424.mp3


Copying audios across to new dataset:   3%| | 1720/55757 [00:06<02:09, 418.71it/

...

Copying audios across to new dataset:  97%|▉| 54305/55757 [01:24<00:01, 848.25it

File copied from /media/kathyreid/Elements/cv-datasets/cv-corpus-23.0-2025-09-05/en/clips/common_voice_en_21024847.mp3 to datasets/commonvoice-v23_en-AU/audio_files/common_voice_en_21024847.mp3


Copying audios across to new dataset:  99%|▉| 55308/55757 [01:25<00:00, 1041.94i

File copied from /media/kathyreid/Elements/cv-datasets/cv-corpus-23.0-2025-09-05/en/clips/common_voice_en_21595227.mp3 to datasets/commonvoice-v23_en-AU/audio_files/common_voice_en_21595227.mp3


Copying audios across to new dataset: 100%|█| 55757/55757 [01:26<00:00, 646.12it


In [46]:
# check how many files are in the destination dir - should match the # of rows in the dataframe 
count = 0; 
with os.scandir(audio_fp_destination) as entries:
    for entry in entries:
        if entry.is_file():
             count += 1

print(count)

55757


## Export the dataframe to dataset.csv

In [36]:
fp = 'datasets/commonvoice-v23_en-AU/commonvoice-v23_en-AU.csv'
filtered_df.to_csv(fp)